# Model Creation and Training

In [9]:
import pandas as pd
import numpy as np
import logging
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Concatenate, Add, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

logging.basicConfig(filename = "logs.log", format = "%(asctime)s -- %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.INFO)

In [10]:
### Load_Data ###
training_data = pd.read_csv("training_data/01Jan2022.csv")
training_data["Datetime"] = training_data["Datetime"].astype("datetime64").astype(int)
training_data.set_index("Datetime", inplace=True)
training_data.head()
percent_positive = (training_data["Response"].sum() / len(training_data)) * 100
X = training_data.loc[:, ["Open", "VWAP"]]
y = training_data.loc[:, "Response"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

logging.info("Dataset Upload successfully")
logging.info(f"X training set shape:{X_train.shape}")
logging.info(f"y training set shape:{y_train.shape}")
logging.info(f"X test set shape:{X_test.shape}")
logging.info(f"y test set shape:{y_test.shape}")
logging.info(f"Percent Positive Response {round(percent_positive,2)}%")
print("Dataset Upload successfully")
print(f"X training set shape:{X_train.shape}")
print(f"y training set shape:{y_train.shape}")
print(f"X test set shape:{X_test.shape}")
print(f"y test set shape:{y_test.shape}")
print(f"Percent Positive Response {round(percent_positive,2)}%")


Dataset Upload successfully
X training set shape:(431216, 2)
y training set shape:(431216,)
X test set shape:(107804, 2)
y test set shape:(107804,)
Percent Positive Response 41.73%


# Batch Data

In [11]:
### Global Variables ####
sequence_len = 1440 # Looking at 24 hours worth of data to determine
epochs = 1
attention_heads = 4
projection_dim = sequence_len + 2
dropout = 0.1
num_transformer_blocks = 2
mlp_units = [2048, 1024]
tranformer_mlp_units = [projection_dim * 2, projection_dim]


In [12]:
batch_size = X_train.shape[0] // sequence_len
max_num_samples = batch_size * sequence_len

X_train = X_train.iloc[:max_num_samples, :]
X_train = tf.expand_dims(X_train, axis = -1)
X_train = np.reshape(X_train, (batch_size, sequence_len, 2))

y_train = y_train.iloc[:max_num_samples]
y_train = tf.expand_dims(y_train, axis = -1)
y_train = tf.expand_dims(y_train, axis = -1)
y_train = np.reshape(y_train, (batch_size, sequence_len, 1))

In [13]:
### Convert time to a vector that can be encoded to the features ###
class Time2Vector(tf.keras.layers.Layer):
    def __init__(self, kernal: int = 64):
        super().__init__(trainable = True,  name = "Time2VecLayer")
        self.kernal = kernal - 1
        
    def build(self, input_shape):
        ### Time to Vector Piecewise function ###
        
        self.weights_linear = self.add_weight(shape = (input_shape[1],1), initializer = "uniform", trainable = True, name = "weights_linear")
        
        self.bias_linear = self.add_weight(shape = (input_shape[1],1), initializer = "uniform", trainable = True, name = "bias_linear")
        
        self.weights_periodic = self.add_weight(shape = (input_shape[1], self.kernal), initializer = "uniform", trainable = True, name = "weights_periodic")
        
        self.bias_periodic = self.add_weight(shape = (input_shape[1], self.kernal), initializer = "uniform", trainable = True, name = "bias_periodic")
    
    def call(self, x):
        x = tf.expand_dims(x[:,:,0], axis = -1)
        time_linear = (self.weights_linear * x) + self.bias_linear
        # time_linear = tf.expand_dims(time_linear, axis = -1) #Expand dimensions to concat later
        
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        # time_periodic = tf.expand_dims(time_periodic, axis = -1)
        
        final_product = tf.concat([time_linear, time_periodic], axis = -1)
        # final_product = tf.expand_dims(final_product, axis = 0)
        return final_product
    
    
test_vector = tf.random.uniform(shape = (1, 1440, 2), dtype = tf.float32)
exampleTV = Time2Vector(1440)(test_vector)
exampleTV.shape
        

TensorShape([1, 1440, 1440])

In [14]:
def mlp_block(x, units):
    x = GlobalAveragePooling1D(data_format = "channels_first")(x)
    for unit in units:
        x = Dense(unit, activation = tf.nn.gelu)(x)
        x = Dropout(0.1)(x)
    return x
def transformer_encoder(inputs, attention_heads, projection_dim, dropout):
    ### Layer Normalization / Multihead Attention Layers ###
    x = LayerNormalization(epsilon = 1e-6)(inputs)
    x = MultiHeadAttention(num_heads = attention_heads, key_dim = projection_dim, dropout = dropout)(x,x)
    skip1 = Add()([x, inputs])
    
    ### Feed Forward ###
    x = LayerNormalization(epsilon = 1e-6)(skip1)
    x = mlp_block(x, tranformer_mlp_units)
    skip2 = Add()([x,skip1])
    
    return skip2


def build_model():
    input = Input(shape = X_train.shape[1:]) # (Batch_size, Sequence Length, Number of Features)
    x = Time2Vector(sequence_len)(input)
    x = Concatenate(axis = -1)([input, x]) 
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, attention_heads, projection_dim, dropout)
    x = mlp_block(x, mlp_units)
    output = Dense(1, activation = "sigmoid")(x)
    
    model = Model(inputs = input, outputs = output)
    model.summary()

    return model

build_model()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1440, 2)]    0           []                               
                                                                                                  
 Time2VecLayer (Time2Vector)    (None, 1440, 1440)   4147200     ['input_3[0][0]']                
                                                                                                  
 concatenate_2 (Concatenate)    (None, 1440, 1442)   0           ['input_3[0][0]',                
                                                                  'Time2VecLayer[0][0]']          
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 1440, 1442)  2884        ['concatenate_2[0][0]']    

In [15]:
def train_model(model):
    optimizer = tf.optimizers.Adam(learning_rate=1e-3, decay = 1e-4)
    checkpoint_path = "/models/"
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = "val_Accuracy", save_best_only = True, save_weights_only = True)
    model.compile(optimizer=optimizer, 
                  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics = [tf.keras.metrics.CategoricalAccuracy(name = "Accuracy")])
    
    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = epochs,
        steps_per_epoch = X_train.shape[0]//batch_size,
        callbacks = [checkpoint_callback],
        
    )
    
    return history

In [19]:
y_train

array([[[1],
        [1],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [0]],

       [[1],
        [1],
        [0],
        ...,
        [1],
        [0],
        [0]],

       ...,

       [[1],
        [0],
        [1],
        ...,
        [0],
        [0],
        [1]],

       [[0],
        [1],
        [1],
        ...,
        [1],
        [0],
        [1]],

       [[0],
        [1],
        [1],
        ...,
        [0],
        [0],
        [1]]])

In [16]:
model = build_model()
training = train_model(model)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 1440, 2)]    0           []                               
                                                                                                  
 Time2VecLayer (Time2Vector)    (None, 1440, 1440)   4147200     ['input_4[0][0]']                
                                                                                                  
 concatenate_3 (Concatenate)    (None, 1440, 1442)   0           ['input_4[0][0]',                
                                                                  'Time2VecLayer[0][0]']          
                                                                                                  
 layer_normalization_12 (LayerN  (None, 1440, 1442)  2884        ['concatenate_3[0][0]']    

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
  